my ai project about magic telescope. Link to data: https://archive.ics.uci.edu/dataset/159/magic+gamma+telescope , more about data in file located: /data/magic04.names

In [29]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [30]:
cols = ["fLength", "fWidth", "fSize", "fConc", "fConc1", "fAsym", "fM3Long", "fM3Trans", "fAlpha", "fDist", "class"]
df = pd.read_csv("../data/magic04.data", names=cols)
df.head()

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [31]:
df["class"] = (df["class"] == "g").astype(int)

In [32]:
# for label in cols[:-1]:
#     plt.hist(df[df["class"] == 1][label], color='blue', label='gamma', alpha=0.7, density=True)
#     plt.hist(df[df["class"] == 0][label], color='red', label='hadron', alpha=0.7, density=True)
#     plt.title(label)
#     plt.ylabel("probablitity")
#     plt.xlabel(label)
#     plt.legend()
#     plt.show()

## spliting data across train, valid, test datastes

In [33]:
train, valid, test = np.split(df.sample(frac = 1), [int(0.7 * len(df)), int(0.85 * len(df))])
print(len(train))
print(len(valid))
print(len(test))

13314
2853
2853


In [34]:
def scale_data(dataframe, oversample=False):
    x = dataframe[dataframe.columns[:-1]].values
    y = dataframe[dataframe.columns[-1]].values
    
    scaler = StandardScaler()
    x = scaler.fit_transform(x)
   
    if oversample:
        ros = RandomOverSampler()
        x, y = ros.fit_resample(x, y)
         
    data = np.hstack((x, np.reshape(y, (-1, 1))))
    
    return data, x, y 

In [35]:
train, x_train, y_train = scale_data(train, oversample=True)
valid, x_valid, y_valid = scale_data(valid, oversample=False)
test, x_test, y_test = scale_data(test, oversample=False)

## k-nearest nighbours

In [36]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [37]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model = knn_model.fit(x_train, y_train)

In [38]:
y_pred = knn_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.71      0.74      1010
           1       0.85      0.88      0.86      1843

    accuracy                           0.82      2853
   macro avg       0.80      0.79      0.80      2853
weighted avg       0.82      0.82      0.82      2853



## Naive Bayes

In [39]:
from sklearn.naive_bayes import GaussianNB

In [40]:
nb_model = GaussianNB()
nb_model = nb_model.fit(x_train, y_train)

In [41]:
y_pred = nb_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.37      0.48      1010
           1       0.72      0.90      0.80      1843

    accuracy                           0.71      2853
   macro avg       0.70      0.64      0.64      2853
weighted avg       0.71      0.71      0.69      2853



## logistic regression

In [42]:
from sklearn.linear_model import LogisticRegression

In [43]:
log_model = LogisticRegression()
log_model = log_model.fit(x_train, y_train)

In [44]:
y_pred = log_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.69      0.68      1010
           1       0.83      0.81      0.82      1843

    accuracy                           0.77      2853
   macro avg       0.75      0.75      0.75      2853
weighted avg       0.77      0.77      0.77      2853



## SVM

In [45]:
from sklearn.svm import SVC

In [46]:
svm_model = SVC()
svm_model = svm_model.fit(x_train, y_train)

In [47]:
y_pred = svm_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.77      0.79      1010
           1       0.88      0.90      0.89      1843

    accuracy                           0.85      2853
   macro avg       0.84      0.83      0.84      2853
weighted avg       0.85      0.85      0.85      2853



## Neural Network

In [53]:
import tensorflow as tf

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [49]:
# # Funkcja do rysowania wykresu straty (loss).
# def plot_loss(history):
#     plt.figure(figsize=(8, 6))
#     plt.plot(history.history['loss'], label='Strata (Loss)')
#     plt.plot(history.history['val_loss'], label='Strata na zbiorze walidacyjnym (Validation Loss)')
#     plt.title('Wykres Straty (Loss)')
#     plt.xlabel('Epoka')
#     plt.ylabel('Strata')
#     plt.legend()
#     plt.show()

# # Funkcja do rysowania wykresu dokładności (accuracy).
# def plot_accuracy(history):
#     plt.figure(figsize=(8, 6))
#     plt.plot(history.history['accuracy'], label='Dokładność (Accuracy)')
#     plt.plot(history.history['val_accuracy'], label='Dokładność na zbiorze walidacyjnym (Validation Accuracy)')
#     plt.title('Wykres Dokładności (Accuracy)')
#     plt.xlabel('Epoka')
#     plt.ylabel('Dokładność')
#     plt.legend()
#     plt.show()

In [50]:
nn_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(10,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

nn_model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='binary_crossentropy',
                metrics=['accuracy'])

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:

In [51]:
with tf.device('/GPU:1'):
    history = nn_model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DestroyResource

KeyboardInterrupt: 

In [ ]:
# plot_loss(history)
# plot_accuracy(history)